# ENVIROMENT
### LANGSMITH

In [2]:
pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# part 1 Overview

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify the variables are loaded (optional)
print(f"LANGCHAIN_TRACING_V2: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"LANGCHAIN_ENDPOINT: {os.getenv('LANGCHAIN_ENDPOINT')}")
print(f"LANGCHAIN_API_KEY: {'*' * 20 if os.getenv('LANGCHAIN_API_KEY') else 'Not set'}")
print(f"OPENAI_API_KEY: {'*' * 20 if os.getenv('OPENAI_API_KEY') else 'Not set'}")

LANGCHAIN_TRACING_V2: true
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com
LANGCHAIN_API_KEY: ********************
OPENAI_API_KEY: ********************


In [4]:
import bs4
from langchain_classic import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

/Users/vansh/Documents/GitHub/Langchain/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps, allowing for easier execution and understanding. It can be achieved through various methods such as simple prompting, task-specific instructions, or relying on external classical planners. The goal is to transform big tasks into manageable components for better problem-solving and planning.'

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps, allowing for easier execution and understanding. It can be achieved through methods such as prompting with specific instructions, human inputs, or utilizing external classical planners for long-horizon planning. Task decomposition helps in transforming big tasks into manageable ones, enhancing model performance and interpretation of the thinking process.'

In [5]:
rag_chain.invoke("What is self reflection? explain in detail")

'Self-reflection is a process that allows autonomous agents to improve by refining past actions and correcting mistakes. It plays a crucial role in real-world tasks where trial and error are common. ReAct integrates reasoning and acting within LLM to enable agents to interact with the environment and generate reasoning traces in natural language.'

# part 2 indexing

In [6]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [7]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [9]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(document_result)

1536

In [10]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806915835035412


In [11]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [14]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [15]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [ ]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [18]:
# Using invoke() - the new method in LangChain v1
docs = retriever.invoke("What is Task Decomposition?")

print(f"Found {len(docs)} relevant documents")

Found 1 relevant documents


# part 4 generation

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [21]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [22]:
# Chain
chain = prompt | llm

In [23]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='Task Decomposition is a technique used by agents to break down complex tasks into smaller and simpler steps, allowing for better planning and execution. It can be achieved through methods such as Chain of Thought and Tree of Thoughts, which involve breaking down tasks into manageable steps and exploring multiple reasoning possibilities at each step. Task decomposition can also be facilitated through simple prompting, task-specific instructions, or human inputs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 315, 'total_tokens': 393, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CkpbHU7EyhweU0uD8grxZdZyaMDjQ', 'service_tier': 'default', 

In [24]:
from langchain_classic import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is self reflection? explain in detail")

"Self-reflection is a process that allows autonomous agents to improve themselves by analyzing and evaluating their past actions, decisions, and mistakes. It involves looking back on one's behavior, thoughts, and experiences in order to learn from them and make adjustments for future actions. Self-reflection is essential for autonomous agents in order to iterate and refine their decision-making processes, ultimately leading to better performance in real-world tasks. \n\nIn the context of ReAct (Yao et al. 2023), self-reflection is integrated into the system by extending the action space to include both task-specific discrete actions and the language space. This allows the autonomous agent to interact with the environment through actions such as using the Wikipedia search API, while also prompting the agent to generate reasoning traces in natural language. The ReAct prompt template provides explicit steps for the agent to engage in self-reflection, including thinking about a situation, 

In [ ]:
"Self-reflection is a process that allows autonomous agents to improve themselves by analyzing and evaluating their past actions, decisions, and mistakes. It involves looking back on one's behavior, thoughts, and experiences in order to learn from them and make adjustments for future actions. Self-reflection is essential for autonomous agents in order to iterate and refine their decision-making processes, ultimately leading to better performance in real-world tasks. \n\nIn the context of ReAct (Yao et al. 2023), self-reflection is integrated into the system by extending the action space to include both task-specific discrete actions and the language space. This allows the autonomous agent to interact with the environment through actions such as using the Wikipedia search API, while also prompting the agent to generate reasoning traces in natural language. The ReAct prompt template provides explicit steps for the agent to engage in self-reflection, including thinking about a situation, taking action based on that thought, observing the outcome, and repeating this process multiple times.\n\nOverall, self-reflection is a crucial aspect of autonomous agents' ability to learn and improve over time, as it allows them to analyze their past actions and make adjustments for future decisions."